In [40]:
import torch
import torch.nn.utils.prune as prune 
import numpy as np
import plotly.express as px
import plotly.graph_objects as go 
import pandas as pd 
import torchvision
from torch.autograd import Variable
import os
from datetime import datetime, timedelta
import yfinance as yf


In [41]:
check_gpu = torch.cuda.is_available()
check_gpu

True

In [42]:
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

In [43]:
check_devices = torch.cuda.get_device_name()
check_devices

'NVIDIA GeForce RTX 2080 Ti'

In [44]:
crypto_name = 'apeusd'
#example_ohlc_df = pd.DataFrame({"Time": [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],"Open": [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],"High": [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],"Low": [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],"Close": [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]})

ohlc_df = pd.read_csv('historical_price/' + crypto_name + '.csv')
ohlc_df.index = pd.to_datetime(ohlc_df['time'], unit = 'ms')

#ohlc_df.index = ohlc_df.index.tz_localize('UTC').tz_convert('US/Eastern')

In [45]:
price_dataset = ohlc_df.close[-1000:]
time_index = ohlc_df.index[-1000:]
print(time_index[-5:])
price_dataset.tail()


DatetimeIndex(['2022-07-19 10:16:00', '2022-07-19 10:18:00',
               '2022-07-19 10:19:00', '2022-07-19 10:20:00',
               '2022-07-19 10:22:00'],
              dtype='datetime64[ns]', name='time', freq=None)


time
2022-07-19 10:16:00    5.7765
2022-07-19 10:18:00    5.7755
2022-07-19 10:19:00    5.7596
2022-07-19 10:20:00    5.7585
2022-07-19 10:22:00    5.7676
Name: close, dtype: float64

In [46]:
price_dataset_x = price_dataset[200:]
time_x = time_index[200:]
price_dataset_y = price_dataset[200:400]
time_y = time_index[200:400]
print(len(price_dataset_y))

200


In [47]:
price_test_x = price_dataset[400:600]
time_test_x = time_index[400:600]
price_test_y = price_dataset[600:900]
time_test_y = time_index[600:900]
price_test_val = price_dataset[900:1000]
time_test_val = time_index[900:1000]

In [48]:
seq_len = 10
batch_size = 55

In [49]:
class TimeSeriesDataset(torch.utils.data.Dataset):
    def __init__(self, X, y, seq_len=5):
        self.X = torch.tensor(X, dtype = torch.float32)
        self.y = torch.tensor(y, dtype = torch.float32)
        self.seq_len = seq_len
    def __len__(self):
        return self.X.__len__() - (self.seq_len - 1)

    def __getitem__(self, index):
        #print(len(self.y))
        X = self.X[index:index+self.seq_len]
        try:
            y = self.y[index+self.seq_len]
        except:
            y = self.y[self.seq_len]
        return X, y

In [50]:
train_dataset = TimeSeriesDataset(price_dataset_x, price_dataset_y, seq_len=seq_len)
test_dataset = TimeSeriesDataset(price_test_x, price_test_y, seq_len=seq_len)
#val_dataset = TimeSeriesDataset(price_test_val, price_test_y, seq_len=seq_len)

In [51]:
train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=batch_size,
    drop_last = True, 
    shuffle=False)

test_loader = torch.utils.data.DataLoader(
    test_dataset, 
    batch_size=batch_size, 
    drop_last=True,
    shuffle=False)

In [52]:
class LSTM_Model(torch.nn.Module):
    def __init__(self, input_dim , hidden_size , num_layers, batch_size):
        super(LSTM_Model, self).__init__()
        self.num_layers = num_layers
        self.input_size = input_dim
        self.hidden_size = hidden_size
        self.batch_size = batch_size
        self.lstm = torch.nn.LSTM(input_size=input_dim , hidden_size = hidden_size , num_layers= num_layers )
        self.fc = torch.nn.Linear(hidden_size,1)

    def forward(self,x,hn,cn):
        out , (hn,cn) = self.lstm(x , (hn,cn))
        final_out = self.fc(out[-1])
        return final_out,hn,cn

    def predict(self,x):
        hn,cn  = self.init()
        final_out = self.fc(hn[-1])
        return final_out

    def init(self):
        h0 =  torch.zeros(self.num_layers , self.batch_size , self.hidden_size)
        c0 =  torch.zeros(self.num_layers , self.batch_size , self.hidden_size)
        return h0 , c0


#device = torch.device('cpu')
#print(device)

input_dim = 1
hidden_size = seq_len
num_layers = 3
crypto_price_model = LSTM_Model(input_dim, hidden_size, num_layers, batch_size)


crypto_price_model#.to(device)

loss_function = torch.nn.MSELoss() # Criterion, I believe CrossEntropyLoss has SoftMax built-in so I used it here, this is why my last layer doesn't have a softmax
optimizer  = torch.optim.Adam(crypto_price_model.parameters(), lr=0.017) # used adaptive moment estimation to optimize the model

In [53]:
#training loop
def train(dataloader, model):
    predictions = []
    loss_list = []
    hn , cn = model.init()
    model.train()
    for batch , item in enumerate(dataloader):
        x , y = item
        y = y.type(torch.FloatTensor)
        #x = x.to(device)
        #y = y.to(device)
        out , hn , cn = model(x.reshape(seq_len,batch_size,-1),hn,cn)
        out = out.view(-1)
        # print(out.shape)
        # print(y.shape)
        loss = loss_function(out.reshape(batch_size) , y)
        hn = hn.detach()
        cn = cn.detach()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        print(f'Training Loss: {loss.item()}')
        if batch == len(dataloader)-1:
            loss = loss.item()
            print(f"Train loss: {loss:>7f} ")
        loss_list.append(loss)
        predictions.append(out.detach().numpy())
        return predictions, loss_list
        

In [54]:
def test(dataloader, model):
    predictions = []
    loss_list = []
    hn , cn = model.init()
    model.eval()
    for batch , item in enumerate(dataloader):
        x , y = item
        y = y.type(torch.FloatTensor)
        #x = x.to(device)
        #y = y.to(device)
        out , hn , cn = model(x.reshape(seq_len,batch_size,1),hn,cn)
        loss = loss_function(out.reshape(batch_size) , y)
        print(f"test loss: {loss.item():>7f} ")
        if batch == len(dataloader)-1:
            loss = loss.item()
            print(f"Test loss: {loss:>7f} ")
        predictions.append(out.detach().numpy())
        loss_list.append(loss)
        return predictions, loss_list

In [55]:
epochs = 100
for epoch in range(epochs):
    print(f"Epoch {epoch + 1}: ")
    train(train_loader, crypto_price_model)
test_predictions = test(test_loader, crypto_price_model)
#print(test_predictions[:][:][:][0])

Epoch 1: 
Training Loss: 31.774856567382812
Epoch 2: 
Training Loss: 30.099302291870117
Epoch 3: 
Training Loss: 28.33785057067871
Epoch 4: 
Training Loss: 26.355173110961914
Epoch 5: 
Training Loss: 24.01873016357422
Epoch 6: 
Training Loss: 21.365036010742188
Epoch 7: 
Training Loss: 18.806747436523438
Epoch 8: 
Training Loss: 16.566299438476562
Epoch 9: 
Training Loss: 14.579123497009277
Epoch 10: 
Training Loss: 12.779038429260254
Epoch 11: 
Training Loss: 11.140388488769531
Epoch 12: 
Training Loss: 9.66594123840332
Epoch 13: 
Training Loss: 8.350886344909668
Epoch 14: 
Training Loss: 7.17581033706665
Epoch 15: 
Training Loss: 6.1217169761657715
Epoch 16: 
Training Loss: 5.173968315124512
Epoch 17: 
Training Loss: 4.319900035858154
Epoch 18: 
Training Loss: 3.5499789714813232
Epoch 19: 
Training Loss: 2.8592920303344727
Epoch 20: 
Training Loss: 2.2464587688446045
Epoch 21: 
Training Loss: 1.7117023468017578
Epoch 22: 
Training Loss: 1.2555007934570312
Epoch 23: 
Training Loss: 0.

In [56]:
forecast_price = np.array([num for num in test_predictions[0][0]]).flatten()
print(forecast_price)

[5.7798786 5.7798786 5.7798786 5.7798786 5.7798786 5.7798786 5.7798786
 5.7798786 5.7798786 5.7798786 5.7798786 5.7798786 5.7798786 5.7798786
 5.7798786 5.7798786 5.7798786 5.7798786 5.7798786 5.7798786 5.7798786
 5.7798786 5.7798786 5.7798786 5.7798786 5.7798786 5.7798786 5.7798786
 5.7798786 5.7798786 5.7798786 5.7798786 5.7798786 5.7798786 5.7798786
 5.7798786 5.7798786 5.7798786 5.7798786 5.7798786 5.7798786 5.7798786
 5.7798786 5.7798786 5.7798786 5.7798786 5.7798786 5.7798786 5.7798786
 5.7798786 5.7798786 5.7798786 5.779879  5.779879  5.779879 ]


In [57]:
print(len(test_predictions[0]))
print()
def ave_percentage_error(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100
#print(ave_percentage_error(price_test_y, forecast_price))

fig = go.Figure()
pred_line = go.Scatter(x=time_test_y[:len(forecast_price)], y=forecast_price, mode = 'lines', name = 'Predicted Price')
fig.add_trace(pred_line)

fig.add_trace(go.Scatter(x=time_test_y[:len(forecast_price)], y=price_test_val[:len(forecast_price)], mode = 'lines', name = 'Actual Price'))

fig.update_layout(xaxis_title="Time", yaxis_title="Actual Price ($)", title=f"Predicted Price vs Actual Price for '{crypto_name}'")
fig.write_json("'LSTM_figures/'+f'{fig_name}_{crypto_name}_predicted_price_test.json")
fig.show()



1



In [84]:

def forecast_prices(most_recent_time, most_recent_data):
    fig = go.Figure()
    forecast = crypto_price_model.predict(most_recent_data).detach().numpy()
    forecast = np.array([num for num in forecast]).flatten()
    td = most_recent_time + timedelta(minutes=len(forecast))
    #print(td)
    forecast_time = [td + timedelta(minutes=i) for i in range(len(forecast))]
    
    pred_line = go.Scatter(x=forecast_time, y=forecast, mode = 'lines', name = 'Predicted Price')

    fig.add_trace(pred_line)
    fig.update_layout(xaxis_title="Time", yaxis_title="Price ($)", title=f"Forecasted Price for '{crypto_name}'")
    #fig.show()
    return fig


In [97]:
def save_figure_as_json(fig, fig_name):
    if not os.path.exists('LSTM_figures'):
        os.mkdir('LSTM_figures')
    fig.write_json('LSTM_figures/'+f'{fig_name}_{crypto_name}.json')
save_figure_as_json(fig, 'test_predictions')

In [93]:
now = datetime.now().replace(second = 0, microsecond=0)
print(now)
ape = yf.download('APE3-USD', period = '2h', interval = '1m')


2022-07-19 17:36:00
[*********************100%***********************]  1 of 1 completed


In [99]:
#print(ape.columns)
#print(ape.index)
ape.index = ape.index.tz_convert('UTC').tz_convert('US/Central')
ape_price = ape['Close']
last_time = ape.index[-1]
print(last_time)
figure = forecast_prices(last_time,ape_price)
save_figure_as_json(figure, 'Forecast')
figure

2022-07-19 17:31:00-05:00


In [95]:
if not os.path.exists('LSTM_parameters'):
    os.mkdir('LSTM_parameters')
#fig.write_html('LSTM_parameters/'+f'{crypto_name}_predicted_price.html')
torch.save(crypto_price_model.state_dict(), f'LSTM_parameters/{crypto_name}_model_weights.pth')

In [96]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
crypto_price_model.load_state_dict(torch.load(f'LSTM_parameters/{crypto_name}_model_weights.pth',))

cuda


<All keys matched successfully>